# Easy Problem

We want to minimize the total cost of deliveries!

In [2]:
import numpy as np
import matplotlib as plt

## Parameters

We have the following problem parameters:
- Cost of the DC in euros $C = 25000$
- Maximum number of DCs $N = 20$
- Expected deliveries per 1000 people $D = 0.2$
- Cost per kilometer of each delivery $K = 1$
- Radius of the Earth $R = 6371.009$

In [23]:
PARAM = {
    "C": 25000,
    "N": 20,
    "D": 0.2,
    "K": 1,
    "R": 6371.009
}

## Data

Open the data we'll work with.

In [25]:
data = np.loadtxt("PopulationCountPT-2020.csv", delimiter=",", dtype=object)

index, name, pop, lat, lon = (data[1:, i] for i in range(5))

pop = pop.astype("float64")
lat = lat.astype("float64")
lon = lon.astype("float64")

# Calculate expected number of deliveries
val = pop * PARAM["D"]


# Solving the Problem

We are going to use the following "greedy" heuristic for solving the problem:
- Calculate the best place to put each DC and put it there!

To implement it, we are going to do the following:
- Compute the distances between every pair of cities and order them from closest to farthest.
- Calculate the value that we get from putting a DC in each available city, supposing each city is supplied by the closest DC to it, and choose the highest.